## 人脸分类大作业
### 无91 2019011559 李天骜

0、准备工作

In [1]:
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #不加这个会报错

1、读取下标集文件

In [2]:
male_file = "male_names.txt"
female_file = "female_names.txt"

male_name = []
female_name = []

with open(male_file) as male_file_object:
    for line in male_file_object:
        line = line[0:-1]#去除换行符
        male_name.append(line)
male_naem = male_name[0:-1]
with open(female_file) as female_file_object:
    for line in female_file_object:
        line = line[0:-1]#去除换行符
        female_name.append(line)
#random.shuffle(male_name)
print(f"There are {len(male_name)} male pictures and {len(female_name)} female pictures.\n")

There are 10268 male pictures and 2966 female pictures.



2、读取图片文件

#将男女图片分开，分别存到两个文件夹中,并在文件名标注性别
train_dir = "/Users/Luke/Jupyter/Summer_Python/photos/"
for name in male_name:
    image =  tf.io.read_file(f"/Users/Luke/Jupyter/Summer_Python/lfw_funneled/{name[0:-9]}/{name}")
    tf.io.write_file(f"{train_dir}male-{name}",image)
    
for name in female_name:
    image =  tf.io.read_file(f"/Users/Luke/Jupyter/Summer_Python/lfw_funneled/{name[0:-9]}/{name}")
    tf.io.write_file(f"{train_dir}female-{name}",image)

In [3]:
#文件名+标签的元组组成examples列表
examples = []
for name in male_name:
    image =  tf.io.read_file(f"/Users/Luke/Jupyter/Summer_Python/male/{name}")
    image_tensor = tf.image.decode_jpeg(image)
    #image_resized = tf.image.resize(image_tensor,[256,256])/255.0
    examples.append((image_tensor,0))
    
for name in female_name:
    image =  tf.io.read_file(f"/Users/Luke/Jupyter/Summer_Python/female/{name}")
    image_tensor = tf.image.decode_jpeg(image)
    #image_resized = tf.image.resize(image_tensor,[265,256])/255.0
    examples.append((image_tensor,1))

print(f"There are {len(examples)} sets of data in total.")

There are 13234 sets of data in total.


3、并随机选择40％、10％、50％的数据，分别作为训练、验证和测试集

In [ ]:
#随机打乱
random.shuffle(examples)

#取50%作为训练样例
train_data = []
train_labels = []
for (data,label) in examples[0:6617]:
    train_data.append(data)
    train_labels.append(label)
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
train_data = tf.convert_to_tensor(train_data, dtype=tf.float32)/255.0
print("train data done")

#取10%作为验证样例
valid_data = []
valid_labels = []
for (data,label) in examples[6617:7917]:
    valid_data.append(data)
    valid_labels.append(label)
valid_labels = tf.convert_to_tensor(valid_labels, dtype=tf.int32)
valid_data = tf.convert_to_tensor(valid_data, dtype=tf.float32)/255.0
print("valid data done")

#取10%作为测试样例
test_data = []
test_labels = []
for (data,label) in examples[7917:-1]:
    test_data.append(data)
    test_labels.append(label)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)
test_data = tf.convert_to_tensor(test_data, dtype=tf.float32)/255.0
print("test data done")

4、训练模型

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2))

model.summary()

In [ ]:
# 训练模型
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_data, train_labels, epochs=10,
                    validation_data=(valid_data, valid_labels))

5、评估模型

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model.evaluate(test_data,test_labels, verbose=2)

In [ ]:
print(test_acc)